In [134]:
# Author:Ning Pei Ding
# Date: 02/10/2020
import pandas as pd
import numpy as np
import scipy.io
import statsmodels.api as sm

In [ ]:
# ?pip install statsmodels

In [135]:
mat = scipy.io.loadmat('china.mat')

/Users/ningpeiding/anaconda3/lib/python3.6/site-packages/scipy/io/matlab/mio.py:142: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [136]:
mret = np.array(mat['china_monthly_ri'])
market = np.array(mat['china_monthly_mv'])
rankrt = np.stack([(mret[:,i] - mret[:,i-1])/mret[:,i-1] for i in range(1,mret.shape[1])], axis = -1)
mk = np.stack([market[:,i] for i in range(1,mret.shape[1])], axis = -1)
momentum = np.stack([np.prod(rankrt[:,i-11:i] + 1,axis = 1) -1 for i in range(11,rankrt.shape[1])], axis = -1)
momentum= np.concatenate((np.zeros([momentum.shape[0],11]), momentum), axis=1)

In [137]:
rf = pd.read_csv('F-F_Research_Data_Factors.CSV')
rf = rf[rf['Time'] >= mat['mdates'][0][0]]['RF']

In [138]:

def percentage(per, data):
    res = []
    for i in per:
        res.append(np.percentile(data,i))
    return res   

In [139]:
def calresult(rankrt, mk, order, percent, rf, start, end, size = 'Full'):
    mtr_mean = []
    vr_mean = []
    ewret = []
    vmret = []
    risk_f = []
    percent = percent
    for i in range(start, end,1):
        rankrt_curr1 = rankrt[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        rankmk1 = mk[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        rankrt_next1 = rankrt[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i]
        rankorder1 =  order[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        if len(rankrt_curr1) == 0 | len(rankorder1) == 0:
            continue
        if size == 'Full':
            rankrt_curr = rankrt_curr1
            rankmk = rankmk1
            rankrt_next = rankrt_next1
            rankorder = rankorder1
        if size == 'Big':
            rankrt_curr = rankrt_curr1[rankmk1 >= np.nanmedian(rankmk1)]
            rankmk = rankmk1[rankmk1 >= np.nanmedian(rankmk1)]
            rankrt_next = rankrt_next1[rankmk1 >= np.nanmedian(rankmk1)]
            rankorder = rankorder1[rankmk1 >= np.nanmedian(rankmk1)]
        if size == 'Small':
            rankrt_curr = rankrt_curr1[rankmk1 < np.nanmedian(rankmk1)]
            rankmk = rankmk1[rankmk1 < np.nanmedian(rankmk1)]
            rankrt_next = rankrt_next1[rankmk1 < np.nanmedian(rankmk1)]
            rankorder = rankorder1[rankmk1 < np.nanmedian(rankmk1)]
        bound = percentage(percent, rankorder) 
        mmean = []
        vr=[]
        mmean.append(np.mean(rankrt_next[rankorder <= bound[0]]))
        vr.append(np.dot(rankrt_next[rankorder <= bound[0]].T , (rankmk[rankorder <= bound[0]]/np.nansum(rankmk))))
        ewret.append(np.mean(rankrt_next))
        vmret.append(np.dot(rankrt_next.T, rankmk/np.nansum(rankmk)))
        risk_f.append(rf.iloc[i])
        if len(bound) > 1:
            for j in range(1,len(bound)):
                mmean.append(np.mean(rankrt_next[(rankorder > bound[j-1]) & (rankorder < bound[j])]))
                vr.append(np.dot(rankrt_next[(rankorder > bound[j-1]) & (rankorder < bound[j])].T, (rankmk[(rankorder > bound[j-1]) & (rankorder < bound[j])]/np.nansum(rankmk))))
            mmean.append(np.mean(rankrt_next[rankorder > bound[-1]]))
            vr.append(np.dot(rankrt_next[rankorder > bound[-1]].T, (rankmk[rankorder > bound[-1]]/np.nansum(rankmk))))
        mtr_mean.append(mmean)
        vr_mean.append(vr)
    mtr_mean = np.array(mtr_mean)
    vr_mean = np.array(vr_mean)
    diff1 = mtr_mean[:,-1] - mtr_mean[:,0]
    diff2 = vr_mean[:,-1] - vr_mean[:,0]
    return ewret, vmret, mtr_mean, vr_mean, diff1, diff2, risk_f

In [140]:
nc = rankrt.shape[1]
nr = rankrt.shape[0]
result = calresult(rankrt,mk,momentum, [20, 40, 60, 80], rf, 87, rankrt.shape[1])

In [141]:
r_minus_rf = np.array([result[2][:,i] - np.array(result[-1])/100 for i in range(result[2].shape[1])])
weightr_minus_rf = np.array([result[3][:,i] - np.array(result[-1])/100 for i in range(result[3].shape[1])])

In [142]:
Rm_minus_Rf = np.array(result[0]) - np.array(result[-1])/100

In [144]:
Weightrm_minus_rf = np.array(result[1]) - np.array(result[-1])/100


# Normal Mean Regression Summary

In [145]:
OLS_m = []
for i in range(len(r_minus_rf)):
    ewresults = sm.OLS(r_minus_rf[i], sm.add_constant(Rm_minus_Rf)).fit()
    OLS_m.append(ewresults)
    print('Part: ',str(i))
    print(ewresults.summary())
    print("-------------------------------------------------------------------\n")  

Part:  0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     4718.
Date:                Mon, 10 Feb 2020   Prob (F-statistic):          1.62e-183
Time:                        23:26:09   Log-Likelihood:                 681.34
No. Observations:                 297   AIC:                            -1359.
Df Residuals:                     295   BIC:                            -1351.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.001      0.159

# The Regression on Difference between Part 1 and Part 5

In [146]:
diffres = sm.OLS(result[4], sm.add_constant(Rm_minus_Rf)).fit()

In [147]:
diffres.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     9.544
Date:                Mon, 10 Feb 2020   Prob (F-statistic):            0.00220
Time:                        23:26:13   Log-Likelihood:                 470.56
No. Observations:                 297   AIC:                            -937.1
Df Residuals:                     295   BIC:                            -929.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0026      0.003     -0.896      0.371      -0.008       0.003
x1            -0.0927      0.030     -3.089      0.002      -0.152      -0.034
==============================================================================
Omnibus:                       11.735   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               24.865
Skew:                          -0.065   Prob(JB):                     3.99e-06
Kurtosis:                       4.412   Cond. No.                         10.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Test on the alpha

In [148]:
print('The difference between part1 alpha and part2 alpha: ', OLS_m[-1].params[0] - OLS_m[0].params[0])

The difference between part1 alpha and part2 alpha:  -0.002618136158240655


In [149]:
print('The alpha of the difference return regression:',diffres.params[0] )

The alpha of the difference return regression: -0.002618136158240655


# Weighted Mean Regression Summary

In [150]:
OLS_wm = []
for i in range(len(r_minus_rf)):
    vmresults = sm.OLS(weightr_minus_rf[i], sm.add_constant(Weightrm_minus_rf)).fit()
    OLS_wm.append(vmresults)
    print('Part: ',str(i))
    print(vmresults.summary())
    print("-------------------------------------------------------------------\n")

Part:  0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     704.6
Date:                Mon, 10 Feb 2020   Prob (F-statistic):           3.69e-80
Time:                        23:26:16   Log-Likelihood:                 956.63
No. Observations:                 297   AIC:                            -1909.
Df Residuals:                     295   BIC:                            -1902.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.001     -2.732

# The Regression on Difference between Part 1 and Part 5

In [151]:
diffres2 = sm.OLS(result[5], sm.add_constant(Weightrm_minus_rf)).fit()
diffres2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     42.80
Date:                Mon, 10 Feb 2020   Prob (F-statistic):           2.68e-10
Time:                        23:26:17   Log-Likelihood:                 752.60
No. Observations:                 297   AIC:                            -1501.
Df Residuals:                     295   BIC:                            -1494.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.001      0.247      0.805      -0.002       0.002
x1             0.0874      0.013      6.542      0.000       0.061       0.114
==============================================================================
Omnibus:                       48.427   Durbin-Watson:                   1.834
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              167.253
Skew:                          -0.656   Prob(JB):                     4.80e-37
Kurtosis:                       6.435   Cond. No.                         12.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Test on the alpha 

In [152]:
print('The difference between part1 alpha and part2 alpha: ', OLS_wm[-1].params[0] - OLS_wm[0].params[0])

The difference between part1 alpha and part2 alpha:  0.00027746192227609914


In [153]:
print('The alpha of the difference return regression:',diffres2.params[0] )

The alpha of the difference return regression: 0.0002774619222760982
